<a href="https://colab.research.google.com/github/skywalker0803r/works/blob/master/bert_dqn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install googletrans==3.1.0a0
#!pip install transformers
#!pip install tensorflow==1.14.0
#!pip install stable-baselines3[extra]

In [2]:
import pandas as pd
df = pd.read_excel('/content/drive/MyDrive/Amazing Gift for Nick1 (1) (1).xlsx')
df

,speak_id,Content,r1score,r2score,r3score,r4score,response1,response2,response3,response4,keywords,intention,emotionlabel,f4,event,entity,relation
0,Student test1,你覺得尼克一開始的反應怎麼樣,1.0,NaN,NaN,NaN,i think nick 's initial reaction was a little ...,What does Nick start to do?,disgust,Who starts to play with the baby?,NaN,SetQ,neutral,Feedback,NaN,NaN,NaN
1,Student test2,"真是過分,怎麼可以欺負小寶寶",0.0,NaN,NaN,NaN,i think it is too much . it is a little bit to...,What do Nick's mom and dad say to the baby?,"Nick smiles and says, I'm a big brother",What kind of robot does the baby crawl to?,NaN,Statement,fear,Nothing,NaN,NaN,NaN
2,Student test1,可能是因為他沒辦法接受吧,0.0,NaN,NaN,NaN,but he does not understand . it is a very smal...,"Who shouts: ""Don't touch it. That's my robot!""",That's not my brother,What does Nick's mom say to the baby?,NaN,Statement,sadness,Feeling,NaN,NaN,NaN
3,Student test2,因為母愛要被分走了?,-1.0,NaN,NaN,NaN,"it is too late , he is only a baby .",What kind of robot does the baby crawl to?,"Nick smiles and says, I'm a big brother",Nick looks back at Nick with what?,NaN,PropQ,surprise,Nothing,NaN,NaN,NaN
4,Student test1,如果是我的話 突然有一個新的人加進我的家庭 我也會覺得很奇怪,0.0,-1.0,NaN,NaN,that is understandable . i hope it is not for ...,What does Nick say to the baby?,I'm a big brother,What does Nick look at the baby with?,NaN,Statement,surprise,Feeling,NaN,NaN,NaN
5,Student test2,太可憐了,NaN,NaN,1.0,NaN,oh no ! that must be so scary .,What does the baby look back at Nick with?,It is so nice to take care of others,What does Nick say the baby points to a toy gi...,NaN,Statement,sadness,Nothing,NaN,NaN,NaN
6,Student test1,"而且還要承擔責任,好慘,你看尼克最一開始還想要把他丟了",1.0,NaN,NaN,NaN,i understand . i am sure he would have been de...,What does Nick want to do to the baby?,It is so nice to take care of others,What does Nick say to the baby?,NaN,Statement,disgust,Feeling,NaN,NaN,NaN
7,Student test2,但是丟到垃圾桶真的太過分了,1.0,NaN,NaN,NaN,that is too bad . i would have been furious,What does Nick say the baby points to a toy gi...,Nick wants to put the baby in a trash can and ...,What does Nick start to do?,NaN,Statement,disgust,Future,NaN,NaN,NaN
8,Student test1,還想了各種方法,-1.0,NaN,NaN,-1.0,"that is good , but there is a lot of work to b...",What do Nick's mom and dad say when they come ...,It is so nice to take care of others,What does Nick say to the baby?,NaN,Statement,neutral,Feeling,NaN,NaN,NaN
9,Student test2,太壞了,-1.0,NaN,-1.0,NaN,that would be a good idea . what would you do ?,What do the baby crawls to Nick and opens his ...,That's not my brother,What does the baby look back at Nick with?,NaN,Statement,disgust,Future,NaN,NaN,NaN


In [3]:
df.columns

Index(['speak_id', 'Content', 'r1score', 'r2score', 'r3score', 'r4score',
       'response1', 'response2', 'response3', 'response4', 'keywords',
       'intention', 'emotionlabel', 'f4', 'event', 'entity', 'relation'],
      dtype='object')

In [4]:
df[['Content', 'r1score', 'r2score', 'r3score', 'r4score',
       'response1', 'response2', 'response3', 'response4',
       'intention', 'emotionlabel', 'f4']]

,Content,r1score,r2score,r3score,r4score,response1,response2,response3,response4,intention,emotionlabel,f4
0,你覺得尼克一開始的反應怎麼樣,1.0,NaN,NaN,NaN,i think nick 's initial reaction was a little ...,What does Nick start to do?,disgust,Who starts to play with the baby?,SetQ,neutral,Feedback
1,"真是過分,怎麼可以欺負小寶寶",0.0,NaN,NaN,NaN,i think it is too much . it is a little bit to...,What do Nick's mom and dad say to the baby?,"Nick smiles and says, I'm a big brother",What kind of robot does the baby crawl to?,Statement,fear,Nothing
2,可能是因為他沒辦法接受吧,0.0,NaN,NaN,NaN,but he does not understand . it is a very smal...,"Who shouts: ""Don't touch it. That's my robot!""",That's not my brother,What does Nick's mom say to the baby?,Statement,sadness,Feeling
3,因為母愛要被分走了?,-1.0,NaN,NaN,NaN,"it is too late , he is only a baby .",What kind of robot does the baby crawl to?,"Nick smiles and says, I'm a big brother",Nick looks back at Nick with what?,PropQ,surprise,Nothing
4,如果是我的話 突然有一個新的人加進我的家庭 我也會覺得很奇怪,0.0,-1.0,NaN,NaN,that is understandable . i hope it is not for ...,What does Nick say to the baby?,I'm a big brother,What does Nick look at the baby with?,Statement,surprise,Feeling
5,太可憐了,NaN,NaN,1.0,NaN,oh no ! that must be so scary .,What does the baby look back at Nick with?,It is so nice to take care of others,What does Nick say the baby points to a toy gi...,Statement,sadness,Nothing
6,"而且還要承擔責任,好慘,你看尼克最一開始還想要把他丟了",1.0,NaN,NaN,NaN,i understand . i am sure he would have been de...,What does Nick want to do to the baby?,It is so nice to take care of others,What does Nick say to the baby?,Statement,disgust,Feeling
7,但是丟到垃圾桶真的太過分了,1.0,NaN,NaN,NaN,that is too bad . i would have been furious,What does Nick say the baby points to a toy gi...,Nick wants to put the baby in a trash can and ...,What does Nick start to do?,Statement,disgust,Future
8,還想了各種方法,-1.0,NaN,NaN,-1.0,"that is good , but there is a lot of work to b...",What do Nick's mom and dad say when they come ...,It is so nice to take care of others,What does Nick say to the baby?,Statement,neutral,Feeling
9,太壞了,-1.0,NaN,-1.0,NaN,that would be a good idea . what would you do ?,What do the baby crawls to Nick and opens his ...,That's not my brother,What does the baby look back at Nick with?,Statement,disgust,Future


In [5]:
import numpy as np
label = df[['r1score', 'r2score', 'r3score', 'r4score']].fillna(-999)
label = pd.get_dummies(label.T.apply(np.argmax)).values
label

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0]], dtype=uint8)

In [6]:
df[['Content','intention', 'emotionlabel', 'f4']]

,Content,intention,emotionlabel,f4
0,你覺得尼克一開始的反應怎麼樣,SetQ,neutral,Feedback
1,"真是過分,怎麼可以欺負小寶寶",Statement,fear,Nothing
2,可能是因為他沒辦法接受吧,Statement,sadness,Feeling
3,因為母愛要被分走了?,PropQ,surprise,Nothing
4,如果是我的話 突然有一個新的人加進我的家庭 我也會覺得很奇怪,Statement,surprise,Feeling
5,太可憐了,Statement,sadness,Nothing
6,"而且還要承擔責任,好慘,你看尼克最一開始還想要把他丟了",Statement,disgust,Feeling
7,但是丟到垃圾桶真的太過分了,Statement,disgust,Future
8,還想了各種方法,Statement,neutral,Feeling
9,太壞了,Statement,disgust,Future


In [7]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")

inputs = ["a", "a"*20, "a"*100, "abcede"*200]
for i in inputs:
  inputs = tokenizer(i, padding='max_length', truncation=True, max_length = 15, return_tensors="pt")
  print(inputs["input_ids"])
  outputs = model(**inputs)
  print(outputs.last_hidden_state.shape, i, len(i))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tensor([[ 101, 1037,  102,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0]])
torch.Size([1, 15, 768]) a 1
tensor([[  101, 13360, 11057, 11057, 11057, 11057, 11057, 11057, 11057, 11057,
          2050,   102,     0,     0,     0]])
torch.Size([1, 15, 768]) aaaaaaaaaaaaaaaaaaaa 20
tensor([[  101, 13360, 11057, 11057, 11057, 11057, 11057, 11057, 11057, 11057,
         11057, 11057, 11057, 11057,   102]])
torch.Size([1, 15, 768]) aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa 100
tensor([[101, 100, 102,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0]])
torch.Size([1, 15, 768]) abcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcede

In [8]:
import googletrans
translator = googletrans.Translator()
results = translator.translate('我覺得今天天氣不好')
print(results)
print(results.text)

Translated(src=zh-CN, dest=en, text=I think the weather is bad today, pronunciation=None, extra_data="{'translat...")
I think the weather is bad today


# preprocess_model

In [106]:
import torch
class preprocess_model(object):
  def __init__(self):
    super().__init__()
    self.translator = googletrans.Translator()
    self.tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    self.bert_model = AutoModel.from_pretrained("bert-base-uncased")

  def __call__(self,sentence):
    x = self.translator.translate(sentence).text #translate
    x = self.tokenizer(x, padding='max_length', truncation=True, max_length = 15, return_tensors="pt") #tokenizer
    x = torch.flatten(self.bert_model(**x).last_hidden_state)[::20]#bert forward and skip
    return x

In [107]:
預處理模型 = preprocess_model()
預處理模型('弟弟不要,brother no').shape

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([576])

In [108]:
x = ''.join(df[['Content','intention', 'emotionlabel', 'f4']].head(1).values[0])
print(x)
display(預處理模型(x))
print(預處理模型(x).shape)

你覺得尼克一開始的反應怎麼樣SetQneutralFeedback


tensor([ 1.0863e-01,  1.0889e-01, -3.9423e-01,  1.4113e-01,  4.0619e-03,
         2.8252e-01, -1.7951e-01, -3.6986e-01,  2.8002e-01, -9.5554e-02,
        -6.7137e-02,  2.5865e-01,  2.0218e-01, -2.0940e-01,  9.6021e-02,
        -1.0215e-01, -8.6413e-03,  7.8537e-02,  1.2302e+00, -1.2610e-01,
         1.8516e-01, -2.5119e-01, -3.1046e-01,  1.1162e-01, -2.6357e-01,
         2.7673e-01, -1.7287e-01,  3.4919e-01, -4.8228e-01,  1.3513e-04,
        -5.2405e-02, -7.3640e-02, -1.3728e-01,  9.4227e-02,  2.0004e-01,
        -4.0513e-02, -1.9421e-01,  3.1257e-01,  2.2402e-01, -4.1681e-01,
        -1.2229e+00, -1.7000e-01, -4.9952e-01,  5.7505e-01, -5.4128e-01,
         5.5216e-01, -4.0635e-02,  9.0129e-01, -1.1059e-01, -3.4088e-01,
         7.4345e-01,  1.1307e-01,  8.2216e-01, -1.6929e-02, -8.7409e-01,
        -2.4016e-01,  1.3861e+00, -7.4665e-01, -9.0687e-01, -8.7373e-02,
         6.1707e-01, -4.0619e-01, -4.7690e-02, -8.4345e-01,  4.5554e-01,
        -3.0066e-01,  1.7794e-01, -8.7999e-02,  6.3

torch.Size([576])


In [109]:
預處理模型(x).min(),預處理模型(x).max()

(tensor(-4.8255, grad_fn=<MinBackward1>),
 tensor(1.7060, grad_fn=<MaxBackward1>))

In [110]:
x = ''.join(df[['Content','intention', 'emotionlabel', 'f4']].head(1).values[0])
x = 預處理模型(x)
x.detach().numpy()

array([ 1.08627260e-01,  1.08894482e-01, -3.94234806e-01,  1.41129240e-01,
        4.06187307e-03,  2.82517910e-01, -1.79506361e-01, -3.69860262e-01,
        2.80016929e-01, -9.55537185e-02, -6.71372786e-02,  2.58646190e-01,
        2.02182800e-01, -2.09396258e-01,  9.60211158e-02, -1.02152832e-01,
       -8.64134263e-03,  7.85371140e-02,  1.23016560e+00, -1.26097918e-01,
        1.85160369e-01, -2.51189262e-01, -3.10457021e-01,  1.11622460e-01,
       -2.63565719e-01,  2.76730955e-01, -1.72868744e-01,  3.49192441e-01,
       -4.82275993e-01,  1.35129274e-04, -5.24045229e-02, -7.36402124e-02,
       -1.37284085e-01,  9.42265987e-02,  2.00039029e-01, -4.05128077e-02,
       -1.94207519e-01,  3.12571973e-01,  2.24016026e-01, -4.16810662e-01,
       -1.22288442e+00, -1.69997886e-01, -4.99517590e-01,  5.75050414e-01,
       -5.41275024e-01,  5.52162766e-01, -4.06345539e-02,  9.01292086e-01,
       -1.10590428e-01, -3.40883106e-01,  7.43447542e-01,  1.13068022e-01,
        8.22162330e-01, -

In [111]:
idx = 0
df[['Content','intention', 'emotionlabel', 'f4']].loc[idx]

Content         你覺得尼克一開始的反應怎麼樣
intention                 SetQ
emotionlabel           neutral
f4                    Feedback
Name: 0, dtype: object

In [112]:
idx = 0
''.join(df[['Content','intention', 'emotionlabel', 'f4']].loc[idx,:].values)

'你覺得尼克一開始的反應怎麼樣SetQneutralFeedback'

# Chatbotenv

In [134]:
import numpy as np
import gym
from gym import spaces

class ChatbotEnv(gym.Env):
  metadata = {'render.modes': ['console']}
  def __init__(self, df):
    super(ChatbotEnv, self).__init__()
    self.idx = 0
    self.features_df = df[['Content','intention', 'emotionlabel', 'f4']]
    self.label_array = df[['r1score', 'r2score', 'r3score', 'r4score']].fillna(-999)
    self.label_array = pd.get_dummies(self.label_array.T.apply(np.argmax)).values
    self.預處理模型 = preprocess_model()
    self.action_space = spaces.Discrete(4)
    self.observation_space = spaces.Box(low=-10,high=10,shape=(576,), dtype=np.float32)
    self.action = 0

  def reset(self):
    self.idx = 0
    state = ''.join(self.features_df.loc[self.idx,:].values)
    state = self.預處理模型(state)
    return state.detach().numpy().reshape(576,)

  def step(self, action):
    self.action = action
    answer = self.label_array[self.idx]
    print(answer)
    if (action == answer).all():
      reward = 1
    else:
      reward = 0
    done = bool(self.idx == len(self.features_df))
    info = {}
    self.idx += 1
    if self.idx >= len(self.features_df):
      self.idx -= 1
    state = ''.join(self.features_df.loc[self.idx,:].values)
    state = self.預處理模型(state)
    return state.detach().numpy().reshape(576,), reward, done, info

  def render(self):
    print('idx:',self.idx)
    print('sentence:',''.join(self.features_df.loc[self.idx,:].values))
    print('action:',self.action)
    print('is correct?',bool((self.action == self.label_array[self.idx]).all()))

  def close(self):
    pass
    

In [135]:
env = ChatbotEnv(df)
env

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [136]:
env.reset()

array([ 1.08627260e-01,  1.08894482e-01, -3.94234806e-01,  1.41129240e-01,
        4.06187307e-03,  2.82517910e-01, -1.79506361e-01, -3.69860262e-01,
        2.80016929e-01, -9.55537185e-02, -6.71372786e-02,  2.58646190e-01,
        2.02182800e-01, -2.09396258e-01,  9.60211158e-02, -1.02152832e-01,
       -8.64134263e-03,  7.85371140e-02,  1.23016560e+00, -1.26097918e-01,
        1.85160369e-01, -2.51189262e-01, -3.10457021e-01,  1.11622460e-01,
       -2.63565719e-01,  2.76730955e-01, -1.72868744e-01,  3.49192441e-01,
       -4.82275993e-01,  1.35129274e-04, -5.24045229e-02, -7.36402124e-02,
       -1.37284085e-01,  9.42265987e-02,  2.00039029e-01, -4.05128077e-02,
       -1.94207519e-01,  3.12571973e-01,  2.24016026e-01, -4.16810662e-01,
       -1.22288442e+00, -1.69997886e-01, -4.99517590e-01,  5.75050414e-01,
       -5.41275024e-01,  5.52162766e-01, -4.06345539e-02,  9.01292086e-01,
       -1.10590428e-01, -3.40883106e-01,  7.43447542e-01,  1.13068022e-01,
        8.22162330e-01, -

In [137]:
env.reset().shape

(576,)

In [138]:
env.step(np.array([1,0,0]))

[1 0 0]


(array([-1.74716756e-01, -2.26626217e-01, -1.83599427e-01, -3.08083352e-02,
         9.11996588e-02,  9.25639793e-02, -1.51168615e-01, -2.72771120e-01,
         1.83654696e-01, -4.71772075e-01, -1.49565712e-02, -1.56244319e-02,
         2.82947570e-01, -7.99918845e-02,  5.04960537e-01,  1.12291589e-01,
        -2.63946056e-02, -7.21021816e-02,  1.19650054e+00, -1.62963495e-01,
        -1.54384956e-01, -6.58554723e-03, -3.31155062e-01,  1.81101799e-01,
        -3.30267757e-01,  2.96117216e-01, -3.36736828e-01,  2.89462805e-01,
        -1.43387645e-01,  8.95399600e-04,  8.60434920e-02,  3.73213053e-01,
        -3.62933874e-01,  3.64701688e-01, -3.53609435e-02,  5.41214645e-03,
         6.95953310e-01,  2.32429132e-01,  3.73101197e-02, -5.65632470e-02,
        -4.32136387e-01,  2.67104656e-01, -5.30164540e-02, -1.64140940e-01,
        -1.64805263e-01,  6.70962036e-01, -8.72946978e-02,  9.37844038e-01,
         1.08121760e-01, -4.10395302e-02,  5.41714668e-01, -2.57884204e-01,
        -8.0

In [139]:
env.render()

idx: 1
sentence: 真是過分,怎麼可以欺負小寶寶StatementfearNothing
action: [1 0 0]
is correct? True


# DQN MODEL

In [146]:
import gym
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines.deepq.policies import MlpPolicy
from stable_baselines import DQN
import warnings
warnings.filterwarnings('ignore')

env = ChatbotEnv(df)

model = DQN(MlpPolicy, env, verbose=1)
model.learn(total_timesteps=10)
model.save("deepq_model")

del model # remove to demonstrate saving and loading

model = DQN.load("deepq_model")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[1 0 0]
[1 0 0]
[1 0 0]
[1 0 0]
[1 0 0]
[0 1 0]
[1 0 0]
[1 0 0]
[1 0 0]
[1 0 0]
Loading a model without an environment, this model cannot be trained until it has a valid environment.


# result

In [147]:
obs = env.reset()
for i in range(30):
  action, _states = model.predict(obs)
  obs, rewards, dones, info = env.step(action)
  env.render()

[1 0 0]
idx: 1
sentence: 真是過分,怎麼可以欺負小寶寶StatementfearNothing
action: 1
is correct? False
[1 0 0]
idx: 2
sentence: 可能是因為他沒辦法接受吧StatementsadnessFeeling
action: 1
is correct? False
[1 0 0]
idx: 3
sentence: 因為母愛要被分走了?PropQsurpriseNothing
action: 1
is correct? False
[1 0 0]
idx: 4
sentence: 如果是我的話 突然有一個新的人加進我的家庭 我也會覺得很奇怪StatementsurpriseFeeling
action: 3
is correct? False
[1 0 0]
idx: 5
sentence: 太可憐了StatementsadnessNothing
action: 1
is correct? False
[0 1 0]
idx: 6
sentence: 而且還要承擔責任,好慘,你看尼克最一開始還想要把他丟了StatementdisgustFeeling
action: 3
is correct? False
[1 0 0]
idx: 7
sentence: 但是丟到垃圾桶真的太過分了StatementdisgustFuture
action: 1
is correct? False
[1 0 0]
idx: 8
sentence: 還想了各種方法StatementneutralFeeling
action: 1
is correct? False
[1 0 0]
idx: 9
sentence: 太壞了StatementdisgustFuture
action: 1
is correct? False
[1 0 0]
idx: 10
sentence: 他還想要把他綁在熱氣球上欸,好慘忍StatementneutralFeeling
action: 3
is correct? False
[0 0 1]
idx: 11
sentence: 那你有一個弟妹會怎樣對待?SetQneutralFuture
action: 1
is correct? False
[1 0 0]
idx: 1